In [2]:
import Globals
import subprocess
import sys

from Containers.AtlasMatrix import AtlasMatrix
from Containers.NodeContainer import NodeContainer
from Containers.AdjacencyMatrix import AdjacencyMatrix
from GUI.ChordGraph import ChordGraph
from IPython.display import display
from ipywidgets import Button, HBox, VBox, SelectMultiple, Dropdown

'''
Creates the buttons to be displayed for Output selection
@returns HBox containing the ToggleButtons(outputs) and Button(redraw)
'''
def create_buttons(output_nodes, traversed_inputs_hash):
    hbutton_list = []
    vbutton_list = []
    
    #update button
    r_button = Button(description='Draw Selection')
    r_button.on_click(redraw)
    hbutton_list.append(r_button)
    
    #output selection menu
    output_select = SelectMultiple(options=output_nodes, description='Outputs: ', disabled=False)
    output_select.observe(change_outputs, 'value')
    hbutton_list.append(output_select)
    
    #input selection menu (region pairs)
    input_select = SelectMultiple(options=input_labels, description='Region pairs\ninputs: ', disabled=False)
    input_select.observe(change_inputs, 'value')
    vbutton_list.append(input_select)
    
    #apply selected input button
    apply_input_select = Button(description='Select Input Region Pairs')
    apply_input_select.on_click(select_in)
    vbutton_list.append(apply_input_select)
    
    vbutton_box = VBox([b for b in vbutton_list])
    hbutton_list.append(vbutton_box)
    
    button_box = HBox([b for b in hbutton_list])
    return button_box
# End create_buttons();


'''
Updates deselected_output_nodes list when selecting outputs from output select widget
'''
def change_outputs(change):
    global deselected_output_nodes
    
    if len(change['new']) == 0: #if non-selected... (selects all)
        deselected_output_nodes = []
    else:
        deselected_output_nodes = [out for out in list(change['owner'].options) if out not in list(change['new'])]
    
# End change_outputs();

'''
Updates deselected_input_nodes list when selecting inputs from input select widget
'''
def change_inputs(change):
    global deselected_input_nodes
    
    to_select = [new_sel for new_sel in change['new'] if new_sel not in change['old']]
    to_deselect = [new_desel for new_desel in change['old'] if new_desel not in change['new']]
    
    if len(change['new']) == 0: #if no selection... (selects all inputs)
        deselected_input_nodes = []
    elif len(change['old']) == 0: #if previous was no selection...
        deselected_input_nodes = [input_labels.index(inpt) + graph_inputs_index for inpt in list(change['owner'].options) if inpt not in list(change['new'])]
    else:
        for selection in to_select:
            deselected_input_nodes.remove(input_labels.index(selection) + graph_inputs_index)
        for deselection in to_deselect:
            deselected_input_nodes.append(input_labels.index(deselection) + graph_inputs_index)
            deselected_input_nodes.sort()
    
    selected_lines = []
    deselected_lines = []
    
    for selection in to_select:
        selected_lines.append(traversed_inputs_hash[selection])
        
    for deselection in to_deselect:
        deselected_lines.append(traversed_inputs_hash[deselection])
    
    atlas.change_node_file_values(selected_lines, 's+2')
    atlas.change_edge_file_values(selected_lines, 0.5)
    
    atlas.change_node_file_values(deselected_lines, 's-2')
    atlas.change_edge_file_values(deselected_lines, 0.1)
    
    #next line has issues when shift-clicking for a multiple
    #line selection.  seems to be that the surf-ice picks up on the shift
    #and it gets added to the ctrl+r, shift+ctrl+r does nothing in surf-ice
    #subprocess.call(['bash','./si_refresh.sh',str(surf_ice.pid)])
    
# End change_inputs();

'''
Forceably updates Surf-Ice with bash script, currently requires 'xdotool' library and X11.
Can get 'xdotool' from aptitude.

TODO: Write script in X11 commands so not to require xdotool library.
'''
def select_in(click):
    subprocess.call(['bash','./si_refresh.sh',str(surf_ice.pid)])
    
# End select_in();

'''
Redraws the chord graph with selected outpits and inputs
'''
def redraw(click):
    chord.remove_edges(deselected_output_nodes, deselected_input_nodes)
    chord.subdraw(connect.get_max_layer(), atlas, True)
    
# End redraw()



############
### MAIN ###
############

# Picks file to run
data = Globals.query_test_file(True)
data.construct_node_container() #contains sorter call, i_nodes.sort(); should test.

# Init. AdjacencyMatrix to generate all graph traversals
container = NodeContainer(data._node_container)
connect = AdjacencyMatrix(container)
output_nodes = [n._node_number for n in connect.get_output_nodes()]
deselected_output_nodes = []
#user_query = Globals.query_outputs(connect.get_output_nodes())

# Creates all paths
connect.generate_all_output_paths() #calls create_paths(self, node, edges_l), depth first traversal filtering unused inputs

# Create Edges

edges = connect.extract_unique_edges(output_nodes) #gives set of edges connected from output(s) to inputs

#would this be needed in non-test cases? 
# edges.sort(key=lambda tup: tup[0])

# Create Atlas Connectivty Matrix, Freatudre Mapping, and .Node file
#leads to surf-ice containing some nodes not connected to output due to using connect.get_input_nodes()
atlas = AtlasMatrix(sys.path[0] + "./Atlas/aal.csv", connect.get_input_nodes())

# Graph Chord Diagram
#TODO: passing connect.get_nodes() passes all nodes, and edges passes
#the unique output path edges; ChordGraph.init_graph() creates more
#vertices than needed before deleting them, could lead to efficiency
#issues with larger data sets.
chord = ChordGraph(connect.get_nodes(), edges, connect.get_max_layer(), atlas)


#TODO: create a function or two for next few definitions
graph_inputs_index = chord._graph.vs['Layer'].index(connect.get_max_layer())

traversed_inputs_hash = {}
i = graph_inputs_index

input_labels = chord._hover_labels[graph_inputs_index:]

for key in input_labels:
    csv_lines = atlas._feature_to_matrix_hash[chord._graph.vs["Node"][i]]
    node_file_indexes = (atlas._l_nodes.index(csv_lines[0]), atlas._l_nodes.index(csv_lines[1]))
    traversed_inputs_hash[key] = node_file_indexes
    i+=1

deselected_input_nodes = []

#Create re-draw buttons
buttons = create_buttons(output_nodes, traversed_inputs_hash)
display(buttons)

chord.draw(True) #takes a bit of time, relative to amount of data or not?

# Displays 3D surfice rendering
#IMPORTANT: SURFICE VERSION ON THE LAB COMP IS OUT OF DATE!
#NEW VERSION SEEMS TO BE BETTER FOR SCRIPTING?
#surf_ice = subprocess.Popen([Globals.SURFICE_DIR,'-S', Globals.SURFICE_SCRIPT])
surf_ice = subprocess.Popen(['/home/drewski/Desktop/Surf_Ice/surficeOld', '-S', '/home/drewski/Desktop/TAKEHOME-COPY/MAY-2019-EDITS/BrianFiles/si_start.py'], stdin =  subprocess.PIPE, stdout = subprocess.PIPE, stderr = subprocess.PIPE)


Which test file? 1) SetOne 2) SetTwo 3) SetThree 4) Test 5) T 5


SEJveChjaGlsZHJlbj0oQnV0dG9uKGRlc2NyaXB0aW9uPXUnRHJhdyBTZWxlY3Rpb24nLCBzdHlsZT1CdXR0b25TdHlsZSgpKSwgU2VsZWN0TXVsdGlwbGUoZGVzY3JpcHRpb249dSdPdXRwdXTigKY=
